In [ ]:
from dataclasses import dataclass
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from basalt.config import DefaultDataConfig
from basalt.linear_probe import LinearProbe
from basalt.config import DefaultDataConfig
import os
import torch
import numpy as np
from dataclasses import dataclass
import tyro
from basalt.common import load_model_parameters
from basalt.vpt_lib.agent import MineRLAgent
from torch.utils.data import DataLoader
import time
import torch_xla.core.xla_model as xm
from basalt.vpt_lib.agent import resize_image, AGENT_RESOLUTION

device = xm.xla_device()
import os
import cv2
import torch
import numpy as np
from torch.utils.data import DataLoader
from concurrent.futures import ProcessPoolExecutor, as_completed
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch_xla


@dataclass
class Args:
    data = DefaultDataConfig()
    task_name: str = "MineRLBasaltFindCave-v0/"


args = Args()

agent_policy_kwargs, agent_pi_head_kwargs = load_model_parameters(args.data.model_path)


def extract_frames(video_path):
    """
    Extract frames at intervals from a video.
    """
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    labels = []

    timesteps = list(range(0, total_frames, 10))

    for timestep in timesteps:
        video.set(cv2.CAP_PROP_POS_FRAMES, timestep)
        success, frame = video.read()

        if success:
            cv2.cvtColor(frame, code=cv2.COLOR_BGR2RGB, dst=frame)
            frame = np.asarray(np.clip(frame, 0, 255), dtype=np.uint8)
            frame = resize_image(frame, AGENT_RESOLUTION)
            frames.append(frame)
            labels.append(timestep / total_frames)
        else:
            print(timestep, video_path)
    video.release()
    return frames, labels


agent = MineRLAgent(
    device=device,
    policy_kwargs=agent_policy_kwargs,
    pi_head_kwargs=agent_pi_head_kwargs,
)
agent.load_weights(args.data.weights_path)
policy = agent.policy
policy.eval()

net = policy.net
video_dir = f"{args.data.task_data_prefix}/{args.task_name}"

linear_probe = LinearProbe(net.hidsize).to(device)




In [ ]:
linear_probe.load_parameters(
    "checkpoints/linear_probe/MineRLBasaltFindCave-v0/epoch_10.pt"
)


In [ ]:
import glob
import random
import torch
import torch_xla
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
dir1_files = random.sample(glob.glob("/data/quangr/demonstrations/MineRLBasaltMakeWaterfall-v0/*.mp4"), 2)
dir2_files = random.sample(glob.glob("/data/quangr/demonstrations/MineRLBasaltFindCave-v0/*.mp4"), 2)
fig, ax = plt.subplots()
label_color_mapping = {
    "dir1": "red",
    "dir2": "blue",
}
handles = []
labels = []

# Process each selected video file
for video_file in dir1_files + dir2_files:
    frames = extract_frames(video_file)[0]
    frames_tensor = torch.tensor(np.array(frames)).to(device)

    with torch.no_grad():
        with torch_xla.step():
            output = net.img_preprocess(frames_tensor[:, None].to(device))
            output = net.img_process(output).squeeze()

    # Get the linear probe output
    linear_probe_output = linear_probe(output).detach().cpu().sigmoid()

    # Determine label based on directory
    if video_file in dir1_files:
        video_label = "dir1"
    else:
        video_label = "dir2"

    # Plot the linear probe output on the same axis with a label
    line,=ax.plot(linear_probe_output, label=video_label, color=label_color_mapping[video_label])
    if video_label not in labels:
        handles.append(line)
        labels.append(video_label)

    # Insert images at indices where linear_probe_output > 0.5
    for i in range(0, len(frames)):
        if linear_probe_output[i] > 0.5:
            img = OffsetImage(frames[i], zoom=0.2)  # Adjust zoom as needed
            ab = AnnotationBbox(img, (i, linear_probe_output[i]), frameon=False)
            ax.add_artist(ab)

# Adjust plot settings
plt.ylim(0, 1)
ax.legend()  # Add a legend to the plot
plt.show()